In [8]:
# This is hardly inspired by the work of Raven Ron on https://www.kaggle.com/codeastar/random-forest-classification-on-talkingdata

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# print(os.listdir("../input"))

import gc, time

dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32'
}

# Correcting path to data inside EMAp servers
path = '/dados/Dados/Kaggle/'

def handleClickHour(df):
    df['click_hour']= (pd.to_datetime(df['click_time']).dt.round('H')).dt.hour
    df['click_hour'] = df['click_hour'].astype('uint16')
    df = df.drop(['click_time'], axis=1)   
    return df

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']

In [12]:
# Importing data

# Load training df (partly)
start_time = time.time()
df_train_30m = pd.read_csv(path + 'train.csv', dtype=dtypes, skiprows=range(1,133333333), nrows=33333333, usecols=train_columns)
print('Load df_train_30m with {} seconds'.format(round(time.time() - start_time, 3)))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 37.9 µs


In [13]:
# Load testing df
start_time = time.time()
df_test = pd.read_csv(path + 'test.csv', dtype=dtypes)
print('Load df_test with {} seconds'.format(round(time.time() - start_time, 3)))

train_record_index = df_train_30m.shape[0]

Load df_test with 61.144 seconds


In [14]:
#handle click hour 
df_train_30m = handleClickHour(df_train_30m)
df_test = handleClickHour(df_test)
gc.collect();
print('ClickTime data correctly handled.')

ClickTime data correctly handled.


In [15]:
#df for submit
df_submit = pd.DataFrame()
df_submit['click_id'] = df_test['click_id']

#Extracting learning data
Learning_Y = df_train_30m['is_attributed']
print('Training target correctly extracted.')

#drop zone
df_test = df_test.drop(['click_id'], axis=1)
df_train_30m = df_train_30m.drop(['is_attributed'], axis=1)
gc.collect();

df_merge = pd.concat([df_train_30m, df_test])
del df_train_30m, df_test
gc.collect();
print('Data was correctly merged')

Training target correctly extracted.
Data was correctly merged


In [16]:
# Count ip for both train and test df 
start_time = time.time()
df_ip_count = df_merge['ip'].value_counts().reset_index(name='ip_count')
df_ip_count.columns = ['ip', 'ip_count']
print('Loaded df_ip_count with {} seconds'.format(round(time.time() - start_time, 3)))

print('Starting to merge with main dataset...')
df_merge = df_merge.merge(df_ip_count, on='ip', how='left', sort=False)
df_merge['ip_count'] = df_merge['ip_count'].astype('uint16')
print('Merging operation completed.')

Loaded df_ip_count with 7.22 seconds
Starting to merge with main dataset...
Merging operation completed.


In [17]:
df_merge = df_merge.drop(['ip'], axis=1)
del df_ip_count
gc.collect();

df_train = df_merge[:train_record_index]
df_test = df_merge[train_record_index:]

del df_merge
gc.collect();

In [18]:
# Start of Random Forest Implementation
from sklearn.ensemble import RandomForestClassifier

print('Starting to fit Random Forest Model... The machine is learning...')
start_time = time.time()
rf = RandomForestClassifier(n_estimators=13, max_depth=13, random_state=13,verbose=2, n_jobs = 18)
rf.fit(df_train, Learning_Y)
print('The machine heas learned.')
print('RandomForest has fitted df_train_30m with {} seconds'.format(round(time.time() - start_time), 3))

Starting to fit Random Forest Model... The machine is learning...
building tree 1 of 13
building tree 2 of 13
building tree 3 of 13
building tree 4 of 13building tree 5 of 13
building tree 6 of 13
building tree 7 of 13
building tree 8 of 13
building tree 9 of 13
building tree 10 of 13
building tree 11 of 13

building tree 12 of 13
building tree 13 of 13


[Parallel(n_jobs=18)]: Done   6 out of  13 | elapsed:  7.7min remaining:  9.0min
[Parallel(n_jobs=18)]: Done  13 out of  13 | elapsed:  8.2min remaining:    0.0s
[Parallel(n_jobs=18)]: Done  13 out of  13 | elapsed:  8.2min finished


The machine heas learned.
RandomForest has fitted df_train_30m with 501 seconds


In [19]:
# Predict
print('Starting prediction phase...')
start_time = time.time()
predictions = rf.predict_proba(df_test)
print('Prediction done. Elapsed time: {} seconds'.format(round(time.time() - start_time, 3)))

Starting prediction phase...


[Parallel(n_jobs=13)]: Done   2 out of  13 | elapsed:    5.9s remaining:   32.6s
[Parallel(n_jobs=13)]: Done   9 out of  13 | elapsed:    6.8s remaining:    3.0s
[Parallel(n_jobs=13)]: Done  13 out of  13 | elapsed:    7.3s finished


Prediction done. Elapsed time: 17.882 seconds


In [21]:
# Creating the submission dataset
df_submit['is_attributed'] = predictions[:,1]
df_submit.describe()
print('Submission dataset created.')

# Preparing submssion
df_submit.to_csv('random_forest_talking_data.csv', index=False)
print('Submission dataset saved correctly.')

df_submit.describe()



,click_id,is_attributed
count,1.879047e+07,1.879047e+07
mean,9.395234e+06,3.618579e-03
std,5.424341e+06,3.498543e-02
min,0.000000e+00,0.000000e+00
25%,4.697617e+06,2.613477e-04
50%,9.395234e+06,3.855908e-04
75%,1.409285e+07,7.654008e-04
max,1.879047e+07,1.000000e+00
